# TopoStats - Minicircle Walk-Through

Welcome, this [Jupyter Notebook](https://jupyter.org/) will take you through processing `minicircle.spm` - a nanoscale AFM height image of DNA atop a flat mica surface using the Python package [TopoStats](https://afm-spm.github.io/TopoStats).

Very little knowledge of [Python](https://www.python.org/) is assumed and attempts have been made to explain every step and where the user might change actions explain how to do so. But if something isn't clear please let us know by creating an [issue](https://github.com/AFM-SPM/jupyterlite-topostats/issues).

## Installing TopoStats

By default the most recent stable release of TopoStats is installed in this JupyterLite Notebook. If however you wish to install the most recent development version directly from Git you should copy and paste the following into a new cell and execute it.

```bash
%pip install git+https://github.com/AFM-SPM/TopoStats.git@main
```




# Setup


## Install requirements

A few libraries need isntalling before you can run this Notebook. This will take a short while as the packages are downloaded and installed.

In [ ]:
%pip install ipywidgets
%pip install topostats

### Loading Libraries and Modules

The required libraries and modules are now `import`ed.

In [ ]:
import json
from pathlib import Path

from ipywidgets import widgets
import matplotlib.pyplot as plt
import numpy as np

from topostats.filters import Filters
from topostats.grains import Grains
from topostats.grainstats import GrainStats
from topostats.io import find_files, read_yaml, write_yaml, LoadScans
from topostats.logs.logs import setup_logger, LOGGER_NAME
from topostats.plottingfuncs import Images
from topostats.tracing.dnatracing import trace_image
from topostats.utils import update_config

In [ ]:
# A composite widget for picking a file and displaying its name
def named_file_upload(accept, description):
    file_upload = widgets.FileUpload(accept=accept, description=description, multiple=False, layout=big_button)
    file_name = widgets.Label(value="No file selected...")

    def handle_file_upload(file):
        file_name.value = f"Selected file : {file['new'][0]['name']}"

    file_upload.observe(handle_file_upload, names="value")
    return widgets.HBox([file_upload, file_name])

# Layout for a bigger button
big_button = widgets.Layout(width="auto")

# Style for wider description
wide_style = {"description_width": "initial"}

# Scan file upload
scan_uploader = named_file_upload(".spm,.gwy", "Upload Scan")

# Config file upload
config_uploader = named_file_upload(".yaml", "Upload Configuration")

## Finding Files

When run from the command line TopoStats needs to find files to process and the `find_files()` function helps here. It
takes as an argument the directory path that should be searched and the file extension to look for (this example uses `.spm`
files) and returns a list of all files in the specified directory which have that file extension
directory. We can use that functionality in this Notebook if you place your files in the same directory as these
Notebooks and execute the next cell.

In [ ]:
# Set the base directory to be current working directory of the Notebook
BASE_DIR = Path().cwd()
# Alternatively if you know where your files area comment the above line and uncomment the below adjust it for your use.
# BASE_DIR = Path("/path/to/where/my/files/are")
# Adjust the file extension approriately.
FILE_EXT = ".spm"
# Search for *.spm files one directory level up from the current notebooks
image_files = find_files(base_dir=BASE_DIR, file_ext=FILE_EXT)

`image_files` is a list of images that match and we can look at that list.

In [ ]:
image_files

## Loading your own file

A sample file, `minicircle.spm` is provided in the `data` directory of this Jupyter Lab page you loaded this Notebook from and that is loaded into the `image_files` list above using `find_files()`. If however you have your own scan you wish to process you can upload it using the button below.

**NB** You _must_ execute the cell first for the button to appear and for the moment only `.spm` files are supported although TopoStats does support other formats.

In [ ]:
## TODO - How to access the selected file as part of TopoStats?
display(scan_uploader)

## Loading a Configuration

You can specify all options explicitly by hand when instantiating classes or calling methods/functions. However when run
at the command line in batch mode TopoStats loads these options from a [YAML](https://yaml.org/) configuration file and it is worth
understanding the structure of this file and how it is used.

A trimmed version is shown below. The words that come before the colon `:` are the option, e.g. `base_dir:` is the base
directory that is searched for files, what comes after is the value, in this case `./tests/`


```yaml
base_dir: ./ # Directory in which to search for data files
output_dir: ./output # Directory to output results to
log_level: info # Verbosity of output. Options: warning, error, info, debug
cores: 2 # Number of CPU cores to utilise for processing multiple files simultaneously.
file_ext: .spm # File extension of the data files.
loading:
  channel: Height # Channel to pull data from in the data files.
filter:
  run: true # Options : true, false
  row_alignment_quantile: 0.5 # below values may improve flattening of larger features
  threshold_method: std_dev # Options : otsu, std_dev, absolute
  otsu_threshold_multiplier: 1.0
  threshold_std_dev:
    below: 10.0 # Threshold for data below the image background
    above: 1.0 # Threshold for data above the image background
  threshold_absolute:
    below: -1.0 # Threshold for data below the image background
    above: 1.0 # Threshold for data above the image background
  gaussian_size: 1.0121397464510862 # Gaussian blur intensity in px
  gaussian_mode: nearest
  # Scar remvoal parameters. Be careful with editing these as making the algorithm too sensitive may
  # result in ruining legitimate data.
  remove_scars:
    run: true
    removal_iterations: 2 # Number of times to run scar removal.
    threshold_low: 0.250 # below values make scar removal more sensitive
    threshold_high: 0.666 # below values make scar removal more sensitive
    max_scar_width: 4 # Maximum thichness of scars in pixels.
    min_scar_length: 16 # Minimum length of scars in pixels.
...
```

For full details of all configuration options please refer to the [configuration](https://afm-spm.github.io/TopoStats/main/configuration.html)
page of the official documentation.

TopoStats comes with a `default_config.yaml` that is loaded by the cell below. This saves the options as a dictionary and
we can access values by the keys. The example below prints out the top-levels keys and then the keys for the `filter`
configuration. 

**NB** Python dictionaries have keys which can be considered as the parameter that is to be configured and each key has
an associated value which is the value you wish to set the parameter to.

In [ ]:
from topostats.io import read_yaml

config = read_yaml("data/default_config.yaml")
print(f"Top level keys of config.yaml : \n\n {config.keys()}\n")
print(f"Configuration options for Filter : \n\n {config['filter'].keys()}")

Alternatively you can use the following button to upload your own configuration file. For convenience a copy of `default_config.yaml` is provided in the `data` directory of this Notebook that you can download and modify before uploading you changes.

In [ ]:
## TODO - How to load file into config?
display(config_uploader)

You can look at all of the options using the `json` package to "pretty" print the dictionary which makes it easier to
read. Here we print the `filter` section. You can see the options map to those of the `Filter()` class with an
additional `"run": true` which is used when running TopoStats at the command line.

In [ ]:
print(json.dumps(config["filter"], indent=4))

We will use the configuration options we have loaded in processing the `minicircle.spm` image. For convenience we save
each set of options to their own dictionary and remove the `run` entry using the `.pop()` method as this is not required when running TopoStats in thos Notebook and would cause errors.

We also set the `plotting_config["image_set"]` to `all` so that all images can be plotted (there are some internal controls that determine whether images are plotted and returned).


In [ ]:
loading_config = config["loading"]
filter_config = config["filter"]
filter_config.pop("run")
grain_config = config["grains"]
grain_config.pop("run")
grainstats_config = config["grainstats"]
grainstats_config.pop("run")
dnatracing_config = config["dnatracing"]
dnatracing_config.pop("run")
plotting_config = config["plotting"]
plotting_config.pop("run")
plotting_config["image_set"] = "all"

## Load Scans

The first step before processing is to load a scan, this extracts the image data to a Numpy array along with the
filepath and the pixel to nanometer scaling parameter which is used to correctly scale the pixels to images. These are
stored in nested dictionaries with one top-level entry for each image that is found.

One of the key fields you may wish to change is the `channel`.

In [ ]:
all_scan_data = LoadScans(image_files, **config["loading"])
all_scan_data.get_data()

# Plot the loaded scan in its raw format
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(all_scan_data.image, cmap="afmhot")
plt.show()

Now that we have loaded the data we can start to process it. The first step is filtering the image.

## Filter Image

Now that we have found some images the first step in processing is to filter them to remove some of the noise. This is
achieved using the `Filters()` class.  There are a number of options that we need to specify which are described in the
table below and also in the [documentation](https://topostats.readthedocs.io/en/dev/topostats.filters.html). 





Once we setup a `Filters` object we can call the different methods that are available for it. There are lots of
different methods that carry out the different steps but for convenience the `filter_image()` method runs all these.

The following section instantiates ("sets up") an object called `filtered_image` of type `Filters` using the first file
found (`image_files[0]`) and the various options from the `filter_config` dictionary.


In [ ]:
from topostats.filters import Filters

filtered_image = Filters(
    image=all_scan_data.img_dict["minicircle"]["image_original"],
    filename=all_scan_data.img_dict["minicircle"]["img_path"],
    pixel_to_nm_scaling=all_scan_data.img_dict["minicircle"]["pixel_to_nm_scaling"],
    row_alignment_quantile=filter_config["row_alignment_quantile"],
    threshold_method=filter_config["threshold_method"],
    otsu_threshold_multiplier=filter_config["otsu_threshold_multiplier"],
    threshold_std_dev=filter_config["threshold_std_dev"],
    threshold_absolute=filter_config["threshold_absolute"],
    gaussian_size=filter_config["gaussian_size"],
    gaussian_mode=filter_config["gaussian_mode"],
    remove_scars=filter_config["remove_scars"],
)


# NB - Because of the one-to-one mapping of configuration options to Filters() options we can use keyword arguments to
#      unpack the options, the below is the same as explicitly stating the values you want to map.
filtered_image = Filters(
    image=all_scan_data.img_dict["minicircle"]["image_original"],
    filename=all_scan_data.img_dict["minicircle"]["img_path"],
    pixel_to_nm_scaling=all_scan_data.img_dict["minicircle"]["pixel_to_nm_scaling"],
    **filter_config,
)
filtered_image.filter_image()

The `filtered_image` now has a a number of NumPy arrays saved in the `.images` dictionary that can be accessed and plotted. To view
the names of the images (technically the dictionary keys) you can print them with `filter_image.images.keys()`...

In [ ]:
print(f"Available NumPy arrays to plot in filter_image.images dictionary :\n\n{filtered_image.images.keys()}")

To plot the raw extracted pixels you can use the built-in NumPy method `imshow()`.


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
plt.imshow(filtered_image.images["gaussian_filtered"], cmap="afmhot")
plt.show()

TopoStats includes a custom plotting class `Images`  which formats plots in a more familiar manner.

It has a number of options, please refer to the official documentation on
[configuration](https://afm-spm.github.io/TopoStats/configuration.html) under the `plotting` entry for what these values
are or the [API
reference](https://afm-spm.github.io/TopoStats/topostats.plottingfuncs.html#module-topostats.plottingfuncs).

The class requires a Numpy array, which we have just generated many of during the various filtering stages, and a number
of options. Again for convenience we use the `**plotting_config` notation to unpack the key/value pairs stored in the
`plotting_config` dictionary.




In [ ]:
fig, ax = Images(
    data=filtered_image.images["gaussian_filtered"],
    filename=filtered_image.filename,
    output_dir="img/",
    save=True,
    **plotting_config,
).plot_and_save()
fig

Here we plot the image after processing and zero-averaging the background but with the `viridis` palette and
constraining the `zrange` to be between 0 and 3

In [ ]:
## TODO - Not getting fig, ax returned by plot_and_save()?
# First remove the current value for cmap in the plotting_config dictionary, otherwise an error occurs because the same
# argument will have been specified twice.
current_cmap = plotting_config.pop("cmap")
current_zrange = plotting_config.pop("zrange")
fig, ax = Images(
    data=filtered_image.images["gaussian_filtered"],
    filename=filtered_image.filename,
    output_dir="img/",
    cmap="viridis",
    zrange=[0, 3],
    save=True,
    **plotting_config,
).plot_and_save()
# Restore the value for cmap to the dictionary.
plotting_config["cmap"] = current_cmap
plotting_config["zrange"] = current_zrange
fig

## Finding Grains

The next step in processing the image is to find grains - a.k.a the molecules we want to analyse. This is done using the `Grains` class and we have saved the
configuration to the `grains_config` dictionary. For details of the arguments and their values please refer to the
[configuration](https://afm-spm.github.io/TopoStats/configuration.html) and the [API
reference](https://afm-spm.github.io/TopoStats/topostats.grains.html).

The most important thing required for grain finding is the resulting image from the Filtering stage, however several
other key variables are required. Again there is a one-to-one mapping between the options to the `Grains()` class and
their values in the configuration file.

The `pixel_to_nm_scaling` is one of several key variables, as is the `filename` and whilst you can specify these
yourself explicitly they are available as properties of the `filtered_image` that we have processed. As with `Filters`
the `Grains` class has a number of methods that carry out the grain finding, but there is a convenience method
`find_grains()` which calls all these in the correct order.

In [ ]:
grains = Grains(
    image=filtered_image.images["final_zero_average_background"],
    filename=filtered_image.filename,
    pixel_to_nm_scaling=filtered_image.pixel_to_nm_scaling,
    threshold_method=grain_config["threshold_method"],
    otsu_threshold_multiplier=grain_config["otsu_threshold_multiplier"],
    threshold_std_dev=grain_config["threshold_std_dev"],
    threshold_absolute=grain_config["threshold_absolute"],
    direction=grain_config["direction"],
    smallest_grain_size_nm2=grain_config["smallest_grain_size_nm2"],
)
# NB - Again we can use the one-to-one mapping of configuration options in the grain_config we extracted.
grains = Grains(
    image=filtered_image.images["final_zero_average_background"],
    filename=filtered_image.filename,
    pixel_to_nm_scaling=filtered_image.pixel_to_nm_scaling,
    **grain_config,
)
grains.find_grains()

The `grains` object now also contains a series of images that we can plot, however, because both `below` and
`above` layers can be processed for grains these reside under the `grains.directions["above"]` and `grains.directions["below"]` dictionaries.

In [ ]:
print(
    f"Available NumPy arrays to plot in grains.directions['above'] dictionary :\n\n{grains.directions['above'].keys()}"
)

And we can again use the `plot_and_save()` function to plot these.

In [ ]:
## TODO - Not getting fig, ax returned by plot_and_save()?
plotting_config["colorbar"] = False
fig, ax = Images(
    data=grains.directions["above"]["coloured_regions"],
    filename=filtered_image.filename,
    output_dir="img/",
    save=True,
    **plotting_config,
).plot_and_save()
fig

### Thresholds

The thresholds can be used in different ways based on the `direction` you want to detect grains. Typically for molecular
imaging where the DNA or protein is raised above the background you will want to look for objects above the surface, using the `above`
threshold. However, when imaging silicon, you may be interested in objects below the surface, using the `below` threshold. For convenience it is
possible to look for grains that are `both` above the `above` threshold and `below` than the below threshold.

This is controlled using the `config["grains"]["direction"]` configuration option which maps to the `Grains(direction=)`
option and can take three values `below`, `above` or `both`.

If you want to change the option you can update the `config["grains"]` dictionary as we do below. You will see that now
we are using `both` there is twice as much output as grains are detected above the reported above Threshold (0.8064)
_and_ below the reported below Threshold (-0.5333).

So far the thresholding method used has been `threshold_method="std_dev"` defined in the configuration file we
loaded. This calculates the mean and standard deviation of height across the whole image and then determines the
threshold by scaling the standard deviation by a given factor (defined by `threshold_std_dev`) and adding it to the mean
to give the `above` threshold and/or subtracting if from the mean to give the `below` threshold.

An alternative method is to use the `threshold_method="absolute"`, set the `direction=above` and explicitly state the
`below` and `above` thresholds (although since we are only looking for objects above a given threshold only the `above`
value will be used). If you wish to specify values for both they are shown below.

In [ ]:
## ToDO - What are sensible values for minicircle here?
grain_config["threshold_method"] = "absolute"
grain_config["direction"] = "above"
grain_config["threshold_absolute"]["above"] = 0.01  # Change just the above threshold
grain_config["threshold_absolute"]["below"] = -2.0  # Change just the below threshold
grain_config["threshold_absolute"] = {
    "below": -2.0,
    "above": 1.2,
}  # Change both the below and above threshold
grains_absolute = Grains(
    image=filtered_image.images["final_zero_average_background"],
    filename=filtered_image.filename,
    pixel_to_nm_scaling=filtered_image.pixel_to_nm_scaling,
    **grain_config,
)
grains_absolute.find_grains()

This is important because you need to know where the resulting images are stored within the `Grains.direction`
dictionary. This will have entries corresponding to the `direction` that grains have been searched for.

In [ ]:
print(f"Grains available in original 'grains' (std_dev, above) : {list(grains.directions.keys())}")
print(f"Grains available in absolute (absolute, above)         : {list(grains_absolute.directions.keys())}")

Each `direction` dictionary is a series of NumPy arrays representing the cleaned images and these can be plotted.

## Grain Statistics

Now that the grains have been found we can calculate statistics for each. This is done using the `GrainStats()`
class. Again the configuration options from the YAML file map to those of the class and there is a convenience method
`calculate_stats()` which will run all steps of grain finding. However, because the class is processing results that we
have generated we have to explicitly pass in more values.

For details of what the arguments are please refer to the [API
reference](https://afm-spm.github.io/TopoStats/topostats.grainstats.html).

The `GrainStats` class returns two objects, a Pandas `pd.DataFrame` of calculated statistics and a `list` of
dictionaries containing the grain data to be plotted. We therefore instantiate ("set-up") the `grain_stats` dictionary
to hold these results and unpack each to the keys `statistics` and `plots` respectively.



In [ ]:
grainstats = GrainStats(
    data=filtered_image.images["gaussian_filtered"],
    labelled_data=grains.directions["above"]["labelled_regions_02"],
    pixel_to_nanometre_scaling=filtered_image.pixel_to_nm_scaling,
    direction="above",
    base_output_dir="grains",
    image_name=filtered_image.filename.stem,
    **grainstats_config,
)
_temp = grainstats.calculate_stats()
grain_stats = {"statistics": _temp[0], "plots": _temp[1]}

The `statistics` is a [Pandas
DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). We can print this out as
shown below.

In [ ]:
grain_stats["statistics"]

Further we can summarise the dataframe or a subset of varaibles.

In [ ]:
grain_stats["statistics"][["smallest_bounding_width", "aspect_ratio"]].describe()

### Plotting Individual Grains

It is possible to plot the individual grains in the same way that whole images are plotted. These are now stored as the
`grain_stats["plots"]` dictionary. We can find out how many grains there are either by looking at the number of rows
reported when the statistics Pandas Data Frame was printed above or its summary (from `.describe()`), which reports the
`count`.

We can then plot any of these using the number indexing of the list. We using
[f-strings](https://realpython.com/python-f-strings/) as the value for the argument `filename` to fill in the images
filename automatically and append the `_grain#` for the image number we are plotting, in this example `0`. Try plotting
some other images.

In [ ]:
fig, ax = Images(
    data=grain_stats["plots"][0]["data"],
    filename=f"{filtered_image.filename}_grain0",
    output_dir="img/",
    save=True,
    **plotting_config,
).plot_and_save()
fig

## DNA Tracing

When working with molecules it is possible to calculate DNA Tracing Statistics using the `dnatracing.py`'s `trace_image` function which takes an image and grain masks, and returns statistics about the dna.


In [ ]:
tracing_results = trace_image(
    image=filtered_image.images["gaussian_filtered"],
    grains_mask=grains.directions["above"]["labelled_regions_02"],
    filename=grains.filename.stem,
    pixel_to_nm_scaling=grains.pixel_to_nm_scaling,
    **dnatracing_config,
)

The results are a dictionary, and the statistics are stored under the `"statistics"` key:

In [ ]:
# Let's have a look at just the tracing stats
tracing_stats = tracing_results["statistics"]
print(tracing_stats)

### Merge GrainStats and TracingStats

Its reasonable to want to have a single data set with which we work and so we now merge the GrainStats data frame with
the Tracing Statistics and then save them to CSV for subsequent use. The following saves them to the directory in which
the Notebook is running with the filename `minicircle_example.csv`

In [ ]:
all_stats_df = grain_stats["statistics"].merge(tracing_stats, on=["image", "molecule_number"])
all_stats_df.to_csv("minicircle_example.csv")

These statistics can now be plotted to show the distribution of the different metrics. Please see the Jupyter Notebook
`notebooks/02-Summary-statistics-and-plots.ipynb` for examples of how to plot these statistics.